In [3]:
import praw

In [4]:
import pickle

In [5]:
import time
import datetime
from calendar import monthrange


In [28]:
import numpy as np
import pandas as pd

In [7]:
reddit = praw.Reddit(client_id='qJwY2ZzRahm-GQ',
                     client_secret='lkZjzt2g6MutrTm3D7c7hNl5R6w',
                     user_agent='linux:reddit-analysis:v1.0.0 (by /u/DigitalArcheology)',
                     username='DigitalArcheology',
                     password='GOvols!2017')

In [8]:
sub = reddit.subreddit('worldnews')

In [11]:
def load_dates(path):
    dates_D = {}
    with open(path, 'r', newline='\n') as f:
        for line in f:
            reddit_id, field = line.strip().split('\t', maxsplit=1)
            dates_D[reddit_id] = field
    return dates_D

def load_data(path, dates_D):
    reddit_ids = []
    data = []
    with open(path, 'r', newline='\n') as f:
        for line in f:
            reddit_id, comments = line.split('\t', maxsplit=1)
            if reddit_id in dates_D.keys():
                data.append(comments)
                reddit_ids.append(reddit_id)
    return reddit_ids, data

In [16]:
dates_D = load_dates('../data/worldnews_2016_10-2017_9_submissiondates.txt')
#reddit_ids, data = load_data('../data/worldnews_2017_1-2017_1_flatcomments.txt', dates_D)

In [17]:
len(dates_D)

9242

In [55]:
timestamp_1Oct2016 = time.mktime(datetime.datetime(2016, 10, 1, 0, 0, 0).timetuple())
timestemp_30Sept2017 = time.mktime(datetime.datetime(2017, 9, 30, 23, 59, 59, 99).timetuple())

In [56]:
with open('../data/worldnews_2016_10-2017_9_contraversial_per_year.txt', 'w') as out:
    for i, submission in enumerate(sub.controversial('year', limit=None)):
        if (submission.num_comments > 100) and (timestamp_1Oct2016 <= submission.created <= timestemp_30Sept2017):
            out.write('%s\t%i\n'%(submission, i))

In [57]:
df = pd.read_csv('../data/worldnews_2016_10-2017_9_contraversial_per_year.txt', sep='\t', header=None, names=['reddit_id', 'rank'], index_col=0); df.head()

,rank
reddit_id,
65k4c1,0
5kcjyi,2
6xaexh,7
5q6aeb,12
5zx9ye,13


In [58]:
df['Date']  = None

In [59]:
for id_ in df.index:
    df.loc[id_, 'Date'] = dates_D[id_]

In [60]:
f = lambda x: pd.Series([int(i) for i in x.split('-')])
rev = df['Date'].apply(f)
rev.rename(columns={0:'Year',1:'Month',2:'Day'},inplace=True)

In [61]:
df = df.join(rev)

In [62]:
groups = df.groupby(by=['Year', 'Month'])

In [66]:
for name, group in groups:
    if name == (2016, 11):
        pass
    else:
        print(group)

           rank        Date  Year  Month  Day
reddit_id                                    
5kcjyi        2  2016-12-26  2016     12   26
5kimkt      111  2016-12-27  2016     12   27
5l0ibu      170  2016-12-30  2016     12   30
           rank        Date  Year  Month  Day
reddit_id                                    
5q6aeb       12  2017-01-25  2017      1   25
5osrs7       22  2017-01-18  2017      1   18
5mjvrk       33  2017-01-07  2017      1    7
5mk7q6       44  2017-01-07  2017      1    7
5p52gx       70  2017-01-20  2017      1   20
5mahkn      100  2017-01-06  2017      1    6
5mfjbi      102  2017-01-06  2017      1    6
5ljpnr      145  2017-01-02  2017      1    2
5n4k19      241  2017-01-10  2017      1   10
5m1bwa      338  2017-01-04  2017      1    4
5nxdbg      649  2017-01-14  2017      1   14
           rank        Date  Year  Month  Day
reddit_id                                    
5skt5b       83  2017-02-07  2017      2    7
5tplbe      456  2017-02-13  2017 

In [65]:
name

(2017, 9)

In [ ]:
for name, group in groups:
    file_name = '../data/worldnews_%i_%i-%i_%i_submissiondates.txt'%(*name, *name)
    group.to_csv(file_name, sep='\t', columns=['Date'], header=False)